# Алгоритмы анализа данных
## Урок 3. Логистическая регрессия. Log Loss

In [1]:
import numpy as np
np.set_printoptions(suppress=True)

In [2]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [3]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [4]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

In [5]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

### Задание 1*

Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.  

**Решение**

In [6]:
def calc_logloss(y, y_pred):
    pred_log1 = np.array([0 if x == 0 else np.log(x) for x in y_pred])
    pred_log2 = np.array([0 if x == 1 else np.log(1.0 - x) for x in y_pred])
    err = -np.mean(y * pred_log1 + (1.0 - y) * pred_log2)
    return err

In [7]:
print(calc_logloss(np.array([0, 1]), np.array([0, 1])))

-0.0


### Задание 2

Подберите аргументы функции `eval_model` для логистической регрессии таким образом, чтобы log loss был минимальным.

**Решение**

In [8]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [9]:
W = eval_model(X_st, y, iterations=500, alpha=4e-2)

50 [ 0.1473526  -0.69332623  0.78048424  1.21237655] 0.5690688150496143
100 [-0.0095343  -0.68398664  0.90804852  1.22410312] 0.5478071912880005
150 [-0.12452451 -0.69193394  0.98395486  1.26905272] 0.5370680049916594
200 [-0.21719855 -0.70785498  1.03124647  1.32969081] 0.529628021007458
250 [-0.29792456 -0.72675622  1.06213101  1.39673902] 0.5234435718105999
300 [-0.37208416 -0.74625081  1.0830114   1.4654754 ] 0.517912031955351
350 [-0.44245931 -0.76522225  1.09731858  1.53359738] 0.5128255960382287
400 [-0.51049676 -0.78317893  1.10695806  1.60004252] 0.5080900785310347
450 [-0.5769575  -0.7999369   1.11304179  1.66437354] 0.5036485765303358
500 [-0.64224857 -0.81546205  1.11626047  1.72646343] 0.49945971986606164


### Задание 3

Создайте функцию `calc_pred_proba`, возвращающую предсказанную вероятность класса 1 (на вход подаются `W`, который уже посчитан функцией `eval_model` и `X`, на выходе - массив `y_pred_proba`).

**Решение**

In [12]:
def calc_pred_proba(W, X):
    z = np.dot(W, X)
    y_pred_proba = sigmoid(z)
    return y_pred_proba

In [13]:
y_pred_proba = calc_pred_proba(W, X_st)
y_pred_proba

array([0.30476172, 0.40991592, 0.65966266, 0.35559873, 0.81119127,
       0.66404101, 0.84136393, 0.22134078, 0.28093523, 0.77509467])

### Задание 4

Создайте функцию `calc_pred`, возвращающую предсказанный класс (на вход подаются `W`, который уже посчитан функцией `eval_model` и `X`, на выходе - массив `y_pred`).

**Решение**

In [14]:
def calc_pred(W, X):
    return np.array([1 if x > 0.5 else 0 for x in calc_pred_proba(W, X)])

In [15]:
y_pred = calc_pred(W, X_st)
y_pred

array([0, 0, 1, 0, 1, 1, 1, 0, 0, 1])

In [16]:
y

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

### Задание 5

Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

**Решение**

In [17]:
accuracy = np.sum([1 if t[0] == t[1] else 0 for t in zip(y, y_pred)]) / len(y)
accuracy

0.8

In [18]:
tp = sum([1 if t[0] == t[1] == 1 else 0 for t in zip(y, y_pred)])
tn = sum([1 if t[0] == t[1] == 0 else 0 for t in zip(y, y_pred)])
fp = sum([1 if t[0] == 1 and t[1] == 0 else 0 for t in zip(y, y_pred)])
fn = sum([1 if t[0] == 0 and t[1] == 1 else 0 for t in zip(y, y_pred)])
print('Confustion matrix')
print('\t1\t0')
print(f'1\t{tp}\t{fp}')
print(f'0\t{fn}\t{tn}')

Confustion matrix
	1	0
1	4	1
0	1	4


In [19]:
precision = tp / (tp + fp)
precision

0.8

In [20]:
recall = tp / (tp + fn)
recall

0.8

In [21]:
f1 = 2 * precision * recall / (precision + recall)
f1

0.8000000000000002

### Задание 6

Могла ли модель переобучиться? Почему?

**Решение**

Модель могла переобучиться, так как в датасете слишком мало наблюдений.

### Задание 7*

Создайте функции `eval_model_l1` и `eval_model_l2` с применением L1 и L2 регуляризаций соответственно.

**Решение**

In [22]:
def eval_model_l1(X, y, iterations, alpha=1e-4, lambda_=1e-8):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T) + lambda_ * np.sign(W))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [23]:
W1 = eval_model_l1(X_st, y, iterations=500, alpha=4e-2, lambda_=6e-4)

50 [ 0.14623823 -0.69233202  0.77919989  1.21126534] 0.5690514325239233
100 [-0.01140632 -0.68220437  0.90540073  1.22210976] 0.5478418934813916
150 [-0.12512872 -0.68972758  0.98011959  1.26600876] 0.5372465319486751
200 [-0.2167357  -0.70521172  1.02651579  1.32549482] 0.5298961131994862
250 [-0.29650218 -0.72367074  1.05669065  1.39137303] 0.5237801607178714
300 [-0.36976422 -0.74273622  1.07698692  1.45895532] 0.5183058052501168
350 [-0.43927958 -0.76129919  1.09080157  1.52595455] 0.513268736101953
400 [-0.5064819  -0.77887056  1.10001972  1.59131432] 0.5085765123366432
450 [-0.57212486 -0.79526654  1.10574026  1.65459871] 0.5041733954752238
500 [-0.63661122 -0.81045214  1.10864492  1.71567987] 0.500018912197697


In [24]:
def eval_model_l2(X, y, iterations, alpha=1e-4, lambda_=1e-8):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T) + lambda_ * W)
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [25]:
W2 = eval_model_l2(X_st, y, iterations=500, alpha=4e-2, lambda_=6e-4)

50 [ 0.14713747 -0.69251674  0.77961989  1.21103517] 0.569100306425443
100 [-0.00965139 -0.68251554  0.90612352  1.22161466] 0.5479320743621374
150 [-0.12438068 -0.68986741  0.98094777  1.26547974] 0.537274429664367
200 [-0.21669735 -0.70521839  1.02720829  1.3250351 ] 0.5298999266581564
250 [-0.29699413 -0.72356415  1.05713517  1.39097893] 0.5237704199966687
300 [-0.37065966 -0.74251538  1.07713531  1.45858349] 0.5182874065813494
350 [-0.44047664 -0.76095582  1.09063736  1.52554691] 0.5132455032831684
400 [-0.50789089 -0.77839481  1.0995431   1.59080925] 0.5085519902301537
450 [-0.57366245 -0.79464926  1.10496087  1.65393579] 0.5041509367424938
500 [-0.63819779 -0.80968571  1.10757833  1.71480165] 0.5000016735246227
